### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [1]:

import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

C:\Users\ASUS\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [12]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '33    Babe (1995)',
 '360    Lion King, The (1994)',
 '2315    Babe: Pig in the City (1998)',
 '1526    Hercules (1997)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [13]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [14]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [15]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [16]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '1892    Rain Man (1988)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

Сделаем сначала с рейтингом тоже самое что и с implicit данными (для rmse)

In [17]:
users = ratings["user_id"]
movies = ratings["movie_id"]
user_item_exp = sp.coo_matrix((ratings["rating"], (users, movies)))

Задание состоит в разделении матрицы на две - юзеры и фильмы, чтобы рекомендовать и находить похожие, мне удобно будет сделать класс, от которого 4 раза можно отнаследоваться

Очевидно нам понадобится rmse и 2 предсказателя

In [18]:
from tqdm import tqdm
from sklearn import metrics
class Splitter:
    def __init__(self, matrix, ratings, k, amount_of_users, amount_of_movies):
        self.ratings = ratings
        self.matrix = matrix
        self.k = k
        self.U = np.random.uniform(0, 1 / np.sqrt(k), (amount_of_users, k))
        self.M = np.random.uniform(0, 1 / np.sqrt(k), (k, amount_of_movies))
        self.B_u = np.zeros((amount_of_users, 1))
        self.B_m = np.zeros((1, amount_of_movies))
        self.mu = ratings.rating.mean() 
    
    def rmse(self):
        return metrics.mean_squared_error(self.matrix.toarray(), np.dot(self.U, self.M), squared=True)
    
    def make_recomendation(self, u_id):
        print('User films')
        print()
        u_films = get_user_history(u_id, self.ratings)[:10]
        print(*u_films, sep='\n')
        print()
        print('Recomendation')
        print()
        prediction = self.U @ self.M
        rating = np.argsort(prediction[u_id])[::-1][:10]
        for i in rating:
            print(*movie_info[movie_info['movie_id'] == i][['name', 'category']].values)
        return
        
    def find_similar(self, m_id):
        print('Film', movie_info[movie_info['movie_id'] == m_id][['name', 'category']].to_string())
        similarity = np.dot(self.M[:,m_id], self.M) / np.sqrt((self.M * self.M).sum(axis=0)) / np.sqrt((self.M[:,m_id] ** 2).sum())
        recomend = np.argsort(similarity)[::-1][:10]
        for i in recomend:
            print(movie_info[movie_info['movie_id'] == i][['name', 'category']].values)
        return

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [27]:
class SVD(Splitter):
    def __init__(self, matrix, ratings, k, amount_of_users, amount_ofmovies):
        super().__init__(matrix, ratings, k, amount_of_users, amount_ofmovies)
    def fit(self, ops, learning_rate = 0.01, regular = 0.001):
        inds = [i for i in range(len(self.ratings))]
        np.random.shuffle(inds)        
        for i in tqdm(range(ops)):
            ind = inds[i % len(self.ratings)]
            u_id = self.ratings.loc[ind]['user_id']
            m_id = self.ratings.loc[ind]['movie_id']
            rat = self.ratings.loc[ind]['rating']
            error = self.U[u_id] @ self.M[:,m_id] + self.B_u[u_id] + self.B_m[:,m_id] + self.mu - rat
            self.U[u_id] = self.U[u_id] - learning_rate * (error * (self.M[:,m_id]).T + regular * self.U[u_id])
            self.M[:,m_id] = self.M[:,m_id] - learning_rate * (error * (self.U[u_id]).T + regular * self.M[:,m_id])
            self.B_u[u_id] = self.B_u[u_id] - learning_rate * (error + regular * self.B_u[u_id])
            self.B_m[:,m_id] = self.B_m[:,m_id] - learning_rate * (error + regular * self.B_m[:, m_id])
            self.mu -= learning_rate * error
            if (i % 100000 == 99999):
                print(self.rmse())

In [37]:
model = SVD(user_item_exp, ratings, 64, max(np.unique(ratings['user_id'])) + 1, max(np.unique(ratings['movie_id'])) + 1)
model.fit(4000000)


  0%|▏                                                                        | 7296/4000000 [00:06<58:57, 1128.72it/s]


  0%|▎                                                                      | 14329/4000000 [00:13<1:30:09, 736.81it/s]


  1%|▍                                                                       | 22287/4000000 [00:20<47:49, 1386.38it/s]


  1%|▌                                                                       | 31589/4000000 [00:27<48:18, 1369.35it/s]


  1%|▋                                                                       | 40991/4000000 [00:34<51:37, 1278.03it/s]


...


  8%|█████▌                                                                 | 310776/4000000 [04:08<36:15, 1695.90it/s]


  8%|█████▌                                                                | 320363/4000000 [04:16<1:04:44, 947.27it/s]


  8%|█████▊                                                                 | 329214/4000000 [04:22<49:49, 1227.74it/s]


  8%|█████▊                                                                 | 330419/4000000 [04:23<48:51, 1251.92it/s]


KeyboardInterrupt: 

In [ ]:
rec = model.make_recomendation(4)

In [ ]:
model.find_similar(1)

In [ ]:
 model.find_similar(10)

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных